# WELCOME TO THE NOTEBOOK  FIFA 20
------------------------------

### Importing the modules

In [ ]:
import numpy as np 
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
import plotly
import re
import os
print("")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print('modules are imported')

### let's load the fifa 2020 dataset

In [ ]:
df_20 = pd.read_csv('/kaggle/input/fifa-20-complete-player-dataset/players_20.csv', error_bad_lines=False)

let's check the dataset

In [ ]:
df_20.head()

checking how many rows and columns we have

In [ ]:
df_20.shape

let's check the columns

In [ ]:
df_20.columns

### Data Preprocessin
Dropping some useless columns

In [ ]:
useless_columns = ['dob','sofifa_id', 'player_url', 'long_name', 'body_type', 'real_face', 
                   'loaned_from', 'nation_position', 'nation_jersey_number']

In [ ]:
df_20  = df_20.drop(useless_columns,axis=1)

Let's check the dataframe again

In [ ]:
df_20.head()

### Calculating BMI 
let's calculate body max index of each Player 

In [ ]:
df_20['BMI'] = df_20['weight_kg'] / (df_20['height_cm'] /100)**2

In [ ]:
df_20.head()

### Player's Position 
Convert the categorical values in Player's Position column in integer values.

In [ ]:
df_20[['short_name','player_positions']]

let's convert the column to integeral columns

In [ ]:
new_player_pos = df_20['player_positions'].str.get_dummies(sep =', ').add_prefix('Position_')
new_player_pos.head()

concatinating new_player_position dataframe to our dataframe

In [ ]:
df_20 = pd.concat([df_20,new_player_pos], axis=1)

let's check the dataset again

In [ ]:
df_20.head()

let's drop the original player_positions columns

In [ ]:
df_20 = df_20.drop('player_positions', axis=1)

let's check the dataset again

In [ ]:
df_20.head()

### Position Columns ratings
Clean, Process and Assign the new attributes to columns listed below.

In [ ]:
columns = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
       'lcb', 'cb', 'rcb', 'rb']

let's check what we have in these columns

In [ ]:
df_20[columns].head()

let's omit the '+' sign

In [ ]:

for col in columns:
    df_20[col] = df_20[col].str.split('+', n=1, expand= True)[0]

df_20[columns].head()

Let's Replace NaN values with 0 

In [ ]:
df_20[columns] = df_20[columns].fillna(0)

Let's convert the columns to int 

In [ ]:
df_20[columns] = df_20[columns].astype(int)

Checking the dataframe again

In [ ]:
df_20[columns].head()

### Filling Missing Values
Let's fill <b>"dribbling", "defending", "physic", "passing", "shooting" and "pace" </b> missing values of these columns by median


In [ ]:
columns2 = ["dribbling", "defending", "physic", "passing", "shooting", "pace"]
df_20[columns2]

At first let's check how many NaN values are there in these columuns

In [ ]:
df_20[columns2].isna().sum()

now lets fill the NaN values with the median of the respective column

In [ ]:
for c in columns2:
    df_20[c] = df_20[c].fillna(df_20[c].median())
df_20[columns2]

let's fill all NaN values in the dataframe with 0

In [ ]:
df_20 =df_20.fillna(0)

let's count the NaN values again 

In [ ]:
df_20.isnull().sum()

## Exploratory Data Analysis

#### 1- Scatter Plot (colored by Age) year 2020 - Overall Rating vs Value in Euros 

In [ ]:
#using plotly
fig = go.Figure(
    
    data = go.Scatter(
        x= df_20['overall'],
        y= df_20['value_eur'],
        mode= 'markers',
        marker = dict(
            size = 10,
            color = df_20['age'],
            showscale = True
        ),
        text = df_20['short_name'])
)
    
fig.update_layout(title=' Scatter Plot ( colored by  age) year 2020 - Overall ratings vs market value in euro'
                  , xaxis_title = 'Overall Rating'
                  , yaxis_title= 'Market Value in Euros')
fig.show()

#### 2- Pie chart proportion of right-foot players vs left-foot players

In [ ]:
fig = px.pie(df_20, names = 'preferred_foot', title= '% of players preferred foot') 
fig.show()

#### 3- Histogram of Players Ages 

In [ ]:
fig = px.histogram(df_20, x= 'age' , title= 'Histogram of Players Ages')
fig.show()

#### 4- Scatterpolar plot to compare a player's grothw over time
let's load the other datasets players from 2015 to 2019

In [ ]:
df_15 = pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_15.csv", error_bad_lines=False)
df_16 = pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_16.csv", error_bad_lines=False)
df_17 = pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_17.csv", error_bad_lines=False)
df_18 = pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_18.csv", error_bad_lines=False)
df_19 = pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_19.csv", error_bad_lines=False)

player attributes column names

In [ ]:
attributes = ['Pace','Shooting','Passing','Dribbling','Defending','Physic','Overall'] 

Creating a method to compare a Players growth over Time

In [ ]:
def playergrowth(name):
    
    data20 = df_20[df_20.short_name.str.startswith(name)]
    data19 = df_19[df_19.short_name.str.startswith(name)]
    data18 = df_18[df_18.short_name.str.startswith(name)]
    data17 = df_17[df_17.short_name.str.startswith(name)]
    data16 = df_16[df_16.short_name.str.startswith(name)]
    data15 = df_15[df_15.short_name.str.startswith(name)]
    
    
    trace0 = go.Scatterpolar(
        
        r = [ data20['pace'].values[0], data20['shooting'].values[0], data20['passing'].values[0]
           , data20['dribbling'].values[0],  data20['defending'].values[0], data20['physic'].values[0]
           , data20['overall'].values[0]]
        ,
        theta = attributes,
        fill = 'toself',
        name = '2020'
    
    )
    trace1 = go.Scatterpolar(
        
        r=[data19['pace'].values[0], data19['shooting'].values[0],
           data19['passing'].values[0] , data19['dribbling'].values[0],  
           data19['defending'].values[0], 
           data19['physic'].values[0]  , data19['overall'].values[0] ],
        theta = attributes,
        fill = 'toself',
        name = '2019'
    
    )
    
    trace2 = go.Scatterpolar(
        
        r = [data18['pace'].values[0], data18['shooting'].values[0], 
             data18['passing'].values[0], data18['dribbling'].values[0], 
             data18['defending'].values[0], data18['physic'].values[0]
           , data18['overall'].values[0] ],
        theta = attributes,
        fill = 'toself',
        name = '2018'
    
    )
    trace3 = go.Scatterpolar(
        
        r  = [ data17['pace'].values[0], data17['shooting'].values[0], data17['passing'].values[0]
           , data17['dribbling'].values[0],  data17['defending'].values[0], data17['physic'].values[0]
           , data17['overall'].values[0] ],
        theta = attributes,
        fill = 'toself',
        name = '2017'
    
    )
    trace4 = go.Scatterpolar(
        
        r = [data16['pace'].values[0], data16['shooting'].values[0], data16['passing'].values[0]
           , data16['dribbling'].values[0],  data16['defending'].values[0], data16['physic'].values[0]
           , data16['overall'].values[0] ],
        theta = attributes,
        fill = 'toself',
        name = '2016'
    
    )
    
    trace5 = go.Scatterpolar(
        
        r = [data15['pace'].values[0], data15['shooting'].values[0], data15['passing'].values[0]
           , data15['dribbling'].values[0],  data15['defending'].values[0], data15['physic'].values[0]
           , data15['overall'].values[0] ],
        theta = attributes,
        fill = 'toself',
        name = '2015'
    
    )
    
    data = [ trace0, trace1, trace2, trace3, trace4, trace5]
    layout = go.Layout(
            polar = dict(
            radialaxis = dict(
            visible = True,
            range = [0,100]))
            , 
            showlegend = True, 
            title = 'Stats retaled to {} from 2015 to 2020'.format(name) )
    
    fig = go.Figure(data = data , layout = layout)
    fig.show()
    
    

Let's check the growth of MSN over time

- **L. Messi**
- **L. Suárez**
-   **Neymar**
![MSN](https://www.fcbarcelonanoticias.com/uploads/s1/11/89/40/8/messi-ney-suarez.jpeg)



In [ ]:
playergrowth('L. Messi')

In [ ]:
playergrowth('L. Suárez')

In [ ]:
playergrowth('Neymar')

What about **BBC**
![BBC](https://footify.fr/image%2Fjpeg/873fe052bff72/16004321190546.jpg)
- **K. Benzema**
- **G. Bale**
- **Cristiano Ronaldo**


In [ ]:
playergrowth('Cristiano Ronaldo')

In [ ]:
playergrowth('K. Benzema')

In [ ]:
playergrowth('G. Bale')

#### 6- Pie chart Describing the Percentage of Players in different Attacker positions

In [ ]:
attack = ['RW', 'LW', 'ST', 'CF', 'LS', 'RS', 'RF', 'LF']

sample_att = df_20.query('team_position in @attack')
sample_att.head()

In [ ]:
fig = px.pie(sample_att, names='team_position', color_discrete_sequence= px.colors.sequential.Magma_r, 
            title = '% of Players in Attacker Position')
fig.show()

#### 7- Pie chart Describing the Percentage of Players in different Midfielder positions

In [ ]:
mid = ['CAM', 'RCM', 'CDM', 'LDM', 'RM', 'LCM', 'LM', 'RDM', 'RAM','CM', 'LAM']

sample_mid = df_20.query('team_position in @mid')
sample_mid.head()

In [ ]:
fig = px.pie(sample_mid, names='team_position', color_discrete_sequence= px.colors.sequential.Blugrn_r ,
            title = '% of Players in MidField Position')
fig.show()

#### 8- Pie chart Describing the Percentage of Players in different Defender positions

In [ ]:
defence = ['LCB', 'RCB', 'LB', 'RB', 'CB', 'RWB', 'LWB']
sample_def = df_20.query('team_position in @defence')
sample_def.head()

In [ ]:
fig = px.pie(sample_def, names='team_position', color_discrete_sequence= px.colors.sequential.GnBu_r, 
            title = '% of Players in Defender Position')
fig.show()

### Pick Top 5 Players per Position
Creating a method to pick top 5 player based on a the player position and the player value in euro

In [ ]:
def pick_top_player(pos, value):
    column = str('Position_')+str.upper(pos)
    target_player= df_20[(df_20[column] == 1) & (df_20['value_eur'] <= value)][['short_name','age','overall','value_eur']].head(5)
    
    return target_player

                        

In [ ]:
pick_top_player('lb',400000000)

In [ ]:
pick_top_player('cf',500000000)

In [ ]:
pick_top_player('ST',55000000)